## オセロのルールを反映させたクラス

In [9]:
# オセロの状態を表すクラス
# 8x8の盤面を持ち、各マスには黒か白か空かのいずれかが入る
# 盤面の状態を配列で表現する
class Board():
    # 8x8の配列を初期化する
    def __init__(self):
        self.board = [[0 for i in range(8)] for j in range(8)]
        self.board[3][3] = 1    # 白(◯)
        self.board[4][4] = 1    # 白
        self.board[3][4] = 2    # 黒(×)
        self.board[4][3] = 2    # 黒
        self.player = 2         # 黒が先手
    # 盤面の配列を取得する
    def getBoard(self):
        return self.board
    # 盤面の状態を表示する
    def show(self):
        for row in self.board:
            print("|", end="")
            for cell in row:
                if cell == 0:
                    print(" ", end="")  # end="" で改行しない
                elif cell == 1:
                    print("o", end="")
                elif cell == 2:
                    print("x", end="")
            print("|")                     # 一行表示して改行
        print("player: o") if self.player == 1 else print("player: x")
    # 置く場所が盤面上にあるか判定する
    def outofBoard(self, x, y):
        return x < 0 or x >= 8 or y < 0 or y >= 8
    # (x, y)に石を置くとき、(dx, dy)方向にひっくり返す石があるか判定する
    def canPutDir(self, x, y, dx, dy):
        i = 1
        if self.outofBoard(x + dx, y + dy):
            return False
        else:
            while self.board[y + dy * i][x + dx * i] == 3 - self.player:    # 相手の石が続く限り
                i += 1
                if self.outofBoard(x + dx * i, y + dy * i):                 # 盤面外に出たら
                    return False
                elif self.board[y + dy * i][x + dx * i] == 0:               # 石がない場所があれば
                    return False
                elif self.board[y + dy * i][x + dx * i] == self.player:     # 自分の石で挟めるなら
                    return True
    # (x, y)に石を置けるか判定する
    def canPut(self, x, y):
        if not self.outofBoard(x,y) and self.board[y][x] != 0:                                           # すでに石が置かれている
            return False
        # 周囲8方向を調べる; 1方向でもひっくり返せるなら置ける
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                if dx == 0 and dy == 0: 
                    continue
                if self.canPutDir(x, y, dx, dy):                             # (dx, dy)方向にひっくり返せる石がある
                    return True
        return False
    # (x, y)に石を置いたときの盤面を返す
    def put(self, x, y):
        newBoard = Board()
        newBoard.board = [row[:] for row in self.board] # 盤面をコピー
        newBoard.player = 3 - self.player   # プレイヤーを交代
        newBoard.board[y][x] = self.player  # 石を置く
        # ひっくり返せる石を全てひっくり返す
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                if dx == 0 and dy == 0:
                    continue
                if self.canPutDir(x, y, dx, dy):
                    i = 1
                    while newBoard.board[y + dy * i][x + dx * i] == 3 - self.player:
                        newBoard.board[y + dy * i][x + dx * i] = self.player    # 石をひっくり返す
                        i += 1
        return newBoard
    # 石を置ける場所があるか判定する
    def canPlay(self):
        for y in range(8):
            for x in range(8):
                if self.canPut(x, y):
                    return True
        return False
    # 石を置ける場所と置いた場合の盤面を辞書型で返す
    def choices(self):          # 動的計画法において、for choice in board.choices(): で呼び出す
        choices = {}
        for y in range(8):
            for x in range(8):
                if self.canPut(x, y):
                    choices[(x, y)] = self.put(x, y)
        return choices
    # 石を置ける場所がなく場合にプレイヤーを交代するか判定し、交代する (交代する場合は新たな盤面を返す)
    def passPlayer(self):   # Board.passPlayer()で呼び出す
        if not self.canPlay():
            # 新たな盤面を作成し、プレイヤーを交代する
            newBoard = Board()
            newBoard.board = [row[:] for row in self.board] # 盤面をコピー
            newBoard.player = 3 - self.player
            return newBoard
    # 勝負がついたか判定する
    def isEnd(self):
        if not self.canPlay() and not self.passPlayer().canPlay():
            print("Game Over")
            return True
        else:
            return False
    # 石の数を数え、勝敗を判定する
    def count(self):
        black = 0
        white = 0
        for row in self.board:
            for cell in row:
                if cell == 1:
                    white += 1
                elif cell == 2:
                    black += 1
        return black, white
    def winner(self):
        black, white = self.count()
        if black > white:
            print("Black(×)")
        elif black < white:
            print("White(◯)")
        else:
            print("Draw")

## 実際にプレイしてみる

In [12]:
from IPython.display import clear_output
import time

# ゲームをプレイする
board = Board()
while not board.isEnd():
    time.sleep(0.5)                                     # 0.5秒待つ
    clear_output()                                      # 画面をクリア
    board.show()                                        # 盤面を表示
    if board.canPlay():
        print("Put coordinate x, y: ", end="")
        x, y = map(int, input().split())                # 入力を受け取る
        print("(", x, ", ", y,")")                      # 入力を表示
        if board.canPut(x, y):
            board = board.put(x, y)
        else:
            print("Can't put")
    else:
        board = board.passPlayer()                      # 石を置けない場合はプレイヤーを交代

|        |
|        |
|        |
|   ox   |
|   xxx  |
|        |
|        |
|        |
player: o
Put coordinate x, y: 

ValueError: not enough values to unpack (expected 2, got 0)

### プレイヤークラスを実装する

プレイヤークラスのメソッドは以下の通りです。
#### リプレイバッファの初期化

#### 報酬の計算

#### リプレイバッファへの追加

実際のプレイヤーの行動とそれに対する報酬を受け取り、リプレイバッファに追加します。

#### 状態行動価値関数の初期化

状態行動価値関数 $Q_{\theta}(s_t, a_t)$ について、初期化する。この際に、マルチステップ方策評価法の、ステップ数 $n_eval$ を指定する。

#### 方策の更新

基本的には、マルチステップ方策更新法に従って、方策を更新する。その際には、Greedy Policy とするか、$\epsilon$-Greedy Policy 、あるいは Softmax Policy とするかを指定する。

#### 行動の選択

自分のターンが来たら、方策に従って行動を選択する。決定論的な方策を取るか、確率的な方策を取るかは、方策の種類次第になる。

## 困難な点

### 状態の表現

全ての盤面を個別の状態として扱うと、状態数が膨大(高々 $3^{64}$ 通り)になる。そのため、状態を適切に抽象化する必要がある。
現時点で、状態行動価値の表現方法(盤面の特徴量)については、以下のような方法が考えられる。
-   自分の置ける場所の種類と数、相手の置ける場所の種類と数の変化
    -   種類は、角、辺(3種)、中央(5種)　の9種類
-   自分の石の数と相手の石の数の変化
-   角を取れる可能性: (対角3列、縦横3列)の状態 の変化

から評価する

あるいは、これらを組み合わせてDNNにより状態行動価値関数を学習することも考えられる。